[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostafaelaraby/wavegan-pytorch/blob/master/notebook.ipynb)


## Cloning Repo and setup of dependencies

In [ ]:
#!git clone https://github.com/mostafaelaraby/wavegan-pytorch.git

In [1]:
import os
os.chdir('./')

In [2]:
!pip3  install -r requirements.txt

## Downloading Data

### Speech Commands Zero through Nine (SC09)

In [4]:
#!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz

--2024-05-29 18:40:47--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-05-29 18:40:52 ERROR 404: Not Found.



In [ ]:
#!tar -xvf sc09.tar.gz

### Drum sound effects

In [5]:
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/drums.tar.gz

--2024-05-29 18:44:17--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/drums.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-05-29 18:44:22 ERROR 404: Not Found.



In [ ]:
!tar -xvf drums.tar.gz

### Bach piano performances

In [ ]:
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/mancini_piano.tar.gz

In [ ]:
!tar -xvf mancini_piano.tar.gz

### Params

In [3]:
%cat params.py
# copy cell content to next cell and edit your params then run 

import torch
import random
import numpy as np
import logging
import os

#############################
# DataSet Path
#############################s
target_signals_dir = "/home/cif/Documents/school/spring2024/trusted_ai/final/wavegan_data/" #"../percept/sorted/beard/" #'piano'
#############################
# Model Params
#############################
model_prefix = 'test' # name of the model to be saved
n_iterations = 100000
lr_g = 1e-4
lr_d = 3e-4 # you can use with discriminator having a larger learning rate than generator instead of using n_critic updates ttur https://arxiv.org/abs/1706.08500
beta1 = 0.5
beta2 = 0.9
use_batchnorm=False
validate=True
decay_lr = False # used to linearly deay learning rate untill reaching 0 at iteration 100,000
generator_batch_size_factor = 1 # in some cases we might try to update the generator with double batch size used in the discriminator https://arxiv.org/abs/1706.08500
n_critic = 1 # update generator every n_critic steps if lr_g = lr_d the n_criti

In [32]:
%%writefile params.py
import torch
import random
import numpy as np
import logging
import os

#############################
# DataSet Path
#############################s
target_signals_dir = "/home/cif/Documents/school/spring2024/trusted_ai/final/wavegan_data/" #"../percept/sorted/beard/" #'piano'
#############################
# Model Params
#############################
model_prefix = 'test' # name of the model to be saved
n_iterations = 100000
lr_g = 1e-4
lr_d = 3e-4 # you can use with discriminator having a larger learning rate than generator instead of using n_critic updates ttur https://arxiv.org/abs/1706.08500
beta1 = 0.5
beta2 = 0.9
use_batchnorm=False
validate=True
decay_lr = False # used to linearly deay learning rate untill reaching 0 at iteration 100,000
generator_batch_size_factor = 1 # in some cases we might try to update the generator with double batch size used in the discriminator https://arxiv.org/abs/1706.08500
n_critic = 1 # update generator every n_critic steps if lr_g = lr_d the n_critic's default value is 5 
# gradient penalty regularization factor.
p_coeff = 10
batch_size = 10
noise_latent_dim = 100  # size of the sampling noise
model_capacity_size = 32*1    # model capacity during training can be reduced to 32 for larger window length of 2 seconds and 4 seconds
# rate of storing validation and costs params
store_cost_every = 300
progress_bar_step_iter_size = 400
#############################
# Backup Params
#############################
take_backup = True
backup_every_n_iters = 1000
save_samples_every = 1000
output_dir = 'output'
if not(os.path.isdir(output_dir)):
    os.makedirs(output_dir)
#############################
# Audio Reading Params
#############################
window_length = 16384 #[16384, 32768, 65536] in case of a longer window change model_capacity_size to 32
sampling_rate = 16000
normalize_audio = True 
num_channels = 1

#############################
# Logger init
#############################
LOGGER = logging.getLogger('wavegan')
LOGGER.setLevel(logging.DEBUG)
#############################
# Torch Init and seed setting
#############################
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# update the seed
manual_seed = 2019 
random.seed(manual_seed)
torch.manual_seed(manual_seed)
np.random.seed(manual_seed)
if cuda:
    torch.cuda.manual_seed(manual_seed)
    torch.cuda.empty_cache()

Overwriting params.py


## Model Training

In [33]:
import librosa.display
import torch
import soundfile as sf
print(torch.cuda.is_available())
torch.cuda.current_device()

!python3  train.py

True
  0%|                                                   | 0/250 [00:00<?, ?it/s]/home/cif/Documents/school/spring2024/trusted_ai/final/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/cif/Documents/school/spring2024/trusted_ai/final/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 20%|▏| 51/250 [05:52<23:05,  6.96s/it, Los